In [1]:
#--- Ameer Shalabi, 2021
# These are functions for parsing (prs) and writing (wrt) for the 
# the Positional Cube Notation (PCN) as well as generating the 
# Shanon cofactors for a given function
#--(if you do not know what PCN is, you should not be here)--
#--- This code made for completion of VLSI CAD Part1 online course.
#--- Please do not use if intended to solve the online assignment. 
#--- Do the assignment yourslef, otherwise you will stay stupid, and
#--- that is no good!
#--- Anyway, the code is shared for the sake of providing functional
#--- free software for someone out there who might need it!
#--- *USE AT YOUR OWN RISK*

In [2]:
# ---- > INPUT and OUTPUT File Format < ---- 
# ---- > THIS IS PART OF THE ONLINE ASSIGNMENT FILE. COPIED VERBATIM.
# ---- > The file is not copies here as plagirism prevention.
# ---- > I am in no way the creator of this format!
# ---- > Example format files from online assignment are provided
# We are using a very simple text file format for this program. 
# Your code will read a Boolean function specified in this format,
# complement this function, and then write out afile in exactly 
# this same file format.
# First line of the file is a number: how many variables in the equation. This is a
# positive integer. We number the variables starting with index 1, so if this number
# was 6, the variables in your problem are: x1, x2, x2, x4, x5, x6. You should
# write your program to handle up to 20 variables. 

# Second line of the file is a number: how many cubes in this cube list. This is a
# positive integer. If there are 10 cubes in this file, this is a “10”. You need to be
# able to handle at least 2^20 different cubes in your program.

# Each of the subsequent lines of the file describes one cube – you have the same
# number of lines as the second line of your file. Each of these lines also has a set
# of numbers: the first number on the line says how many variables are not don’t
# cares in this cube. If this number is, e.g., 5, then the next 5 numbers on the line
# specify the true or complemented form of each variable in this cube. We use a
# simple convention: if variable xk appears in true form, then put integer “k” on
# the line; if variable xk appears in complement form (xk’) then put integer “-k”
# on the line. The file will always order these variables in increasing index order.
# So, if your cube has (x3x5x9’), the line should say “3 3 5 -9” and not
# some other order, e.g., “3 -9 3 5”. Spaces on the line do not matter.

# That’s it. This is really very simple. Suppose we had this function as input:
# F(x1, x2, x3, x4, x5, x6) =x2x4x5’ + x2’x4’x6 + x1x2x3’x4’ + x5x6
# Then the input file format would look like this:
# ---- > INPUT FILE
    # 6
    # 4
    # 3 2 4 -5
    # 3 -2 -4 6
    # 4 1 2 -3 -4
    # 2 5 6
    
# Note: to keep things simple, your output file has exactly this same format. Also, we
# promise not to try to be “tricky” for the required part of this assignmnt, and so we will
# not give you either the Boolean function F=1, or F=0, as inputs. Just to be clear, F=1 has
# just one cube in it, but with only don’t cares in this cube. If F is a function of 6 variables
# like our previous example, the function F(x1,x2,x3,x4,x5,x6)=1 is this file:
# ---- >INPUT FILE
    # 6
    # 1
    # 0
# And, we promise not to give you the input function F=0, which is just an empty list with
# no cubes in it. F(x1,x2,x3,x4,x5,x6)=0 is this file:
# ---- >INPUT FILE
    # 6
    # 0
# It’s not hard to detect these as special cases and return the trivial complement, so we will
# just not do this case, to focus on the interesting part of the algorithm. But, we can give
# you any other function, of up to 20 variables, with up to 2^20 cubes, as input

In [3]:
# Load important libraries:
# OS lib because that is useful
import os

# if islice() and chain() functions are no loaded at kernel start, load the itertools lib
from itertools import *

# create valid path function to check if directory is valid
def valid_path(path=""):
    return os.path.isdir(path)
# create valid path function to check if file is valid
def valid_file(path=""):
    return os.path.isfile(path)

In [4]:
def PCN_prs(pcn_file_path=""):
    # this is a function that parses a pcn file. It returns two integers and a tuple
    # Integers are vraiable count (varCount) and number of cubes (cubCount).
    # tuple contains all the cubes (cubTuple)
    # if file does not exist, the function will return None
    if valid_file(pcn_file_path):
        # open file if valid path is given
        with open(pcn_file_path, "rb") as pcn_file:
            # convert file lines to iterate over
            fileLines = iter(pcn_file)
            # first line is number of variables
            varCount = int(next(fileLines))
            # second line is number of cubes
            cubCount = int(next(fileLines)) 
            # create an array of cubes
            cubArray = [None]*cubCount
            
            # extract all cubes from the rest of the file lines in this for loop
            for cube in range(cubCount):
                # get the next line until end of file!
                line = next(fileLines)
                # splite the line into elements and conver them to integers to
                # represent the variables
                elements=map(int, line.split())
                # create tuple list of the elements
                # add the tuple list to the cube list
                cubArray[cube] = tuple(islice(elements, 1, None))
                # Repeat until no more lines
            cubTuple = tuple(cubArray)
        pcn_file.close() # close the read file when done
        out_str = "Output : \n --> Number of Variables: " + str(varCount) + "\n --> Number of Cubes: " + str(cubCount) + "\n"
        out_str = out_str + " --> Origional Cubes: \n" 
        for item in cubArray:
            out_str = out_str +  "\t" + str(item) + "\n"
            
        print(out_str)
        return (varCount, cubCount, cubTuple)
    else:
        print('"'+pcn_file_path+'"' + " is not a valid path")
        return None


In [5]:
def generateCofactors(cubes=(None),var=0):
    # generate the positive (pos) or negative (neg) cofactor (cofactor) 
    # of variable (var) in the cubes (cubes)
    # get the variables absulute value (in case the var was negative)
    var=abs(var)
    # make a list for access!
    cubes=list(cubes)
    # create a list of both the ositive and negative cofactor cubes
    pos=[]
    neg=[]
    # for every cube in the cube list, check for variable or complement
    # of variable
    for cube in cubes:
        # if the complement of variable is not in a cube (the cube remains 
        # for positive cofactor)
        if -var not in cube:
            posCofactorCube=[]
            # check each element in the cube
            for c in cube:
                # only keep the variables that are not variable being 
                # cofactored
                if c != var:
                    posCofactorCube.append(c)
            # add the cube to the positive cofactor list
            pos.append(tuple(posCofactorCube))
        
        # if the variable is not in a cube (the cube remains for negative 
        # cofactor)
        if var not in cube:
            negCofactorCube=[]
            # check each element in the cube
            for c in cube:
                # only keep the variables that are not variable being 
                # cofactored
                if c != -var:
                    negCofactorCube.append(c)
            # add the cube to the negative cofactor list
            neg.append(tuple(negCofactorCube))
    # return the positive and negative cofactor lists as tuples
    return tuple(pos),tuple(neg)


In [7]:
def PCN_wrt(varCount,cubes,out_file_path="", out_file_name="PCN_wrt_out.pcn"):
    # must give valid path for output file
    if not valid_path(out_file_path):
        print("Output path not valid")
        return None
    else:
        # if the output path is valid, but does not end wtih bachslash, add it
        if out_file_path[-1]!="/":
            out_file_path=out_file_path+"/"
    # check if file name ends with ".pcn", if not, add it
    if out_file_name[-4:]!=".pcn":
        out_file_name=out_file_name+".pcn"
    
    # put full path and file name together
    outfile=out_file_path+out_file_name
    # open file for writing, this will also create file if it does not exist
    with open(outfile, "w") as pcn_out:
        pcn_out.write(str(varCount)+"\n") # write the number of variables to file
        pcn_out.write(str(len(cubes))+"\n") # write the number of cubes to file
        # create a set from the cube tuple to iterate over and write to file
        tuplSet = set(cubes)
        for newCubes in tuplSet:
            # the tuple string has ',' and '(' and ')' chars in it. That is no bueno 
            # for output file replace each of those chars with an empty char. Replaced
            # one char at a time, but if someone comes up with a better way than this,
            # please be my guset and do it. I am content with this method!
            tuple_str = str(newCubes).replace(',', '').replace('(','').replace(')','')
            # write the length of the cube and the cube variables to the output file!
            pcn_out.write(str(len(newCubes))+" "+ str(tuple_str)+"\n")
    print("Writing PCN file to "+outfile+" is done.")



In [17]:
### Example run on part2.pcn with variable 1
varCount, cubCount, cubTuple = PCN_prs(pcn_file_path="part2.pcn")
TuplePos,TupleNeg = generateCofactors(cubTuple,1)
print(TupleNeg)
print(TuplePos)
PCN_wrt(varCount,TuplePos,out_file_path="./",out_file_name="PCN_out_pos")
PCN_wrt(varCount,TupleNeg,out_file_path="./",out_file_name="PCN_out_neg")

Output : 
 --> Number of Variables: 6
 --> Number of Cubes: 8
 --> Origional Cubes: 
	(1, 3, -5)
	(3, 4, 5, -6)
	(-1, 2, 6)
	(1, -2, -3, 4, 5)
	(-4, -5, 6)
	(1, 4, 6)
	(2, 3, -5)
	(1, 2, 3, 4)

((3, 4, 5, -6), (2, 6), (-4, -5, 6), (2, 3, -5))
((3, -5), (3, 4, 5, -6), (-2, -3, 4, 5), (-4, -5, 6), (4, 6), (2, 3, -5), (2, 3, 4))
Writing PCN file to ./PCN_out_pos.pcn is done.
Writing PCN file to ./PCN_out_neg.pcn is done.
